<a href="https://colab.research.google.com/github/JPA-BERT/jpa-bert.github.io/blob/master/notebooks/2020_0722word_embeddings_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch tutorial の内容について(3)

- date: 2020-0722
- author: 浅川伸一

[https://github.com/pytorch/tutorials/tree/master/beginner_source/nlp](https://github.com/pytorch/tutorials/tree/master/beginner_source/nlp) を見ると
PyTorch で 自然言語処理を行う場合のチュートリアルは以下とおりである

# Deep Learning for NLP with Pytorch

1. [pytorch_tutorial.py](https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/pytorch_tutorial.py): 
	[PyTorch 入門 Introduction to PyTorch](https://pytorch.org/tutorials/beginner/nlp/pytorch_tutorial.html)

2. [deep_learning_tutorial.py](https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/deep_learning_tutorial.py): 
	[PyTorch による深層学習 Deep Learning with PyTorch](https://pytorch.org/tutorials/beginner/nlp/deep_learning_tutorial.html)

3. [word_embeddings_tutorial.py](https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/word_embeddings_tutorial.py): 
	[単語埋め込み:語彙的意味の符号化 Word Embeddings: Encoding Lexical Semantics](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)

4. [sequence_models_tutorial.py]((https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/sequence_models_tutorial.py): 
	[系列モデルと LSTM Sequence Models and Long-Short Term Memory Networks](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html)

5. [advanced_tutorial.py]((https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/advanced_tutorial.py): 
	[動的意思決定と双方向 LSTM 条件付き確率場 Advanced: Making Dynamic Decisions and the Bi-LSTM CRF](https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html)


以下では，このうちの 3 について解説している。


In [1]:
%matplotlib inline

<!--
# Word Embeddings: Encoding Lexical Semantics

Word embeddings are dense vectors of real numbers, one per word in your vocabulary. 
In NLP, it is almost always the case that your features are words! 
But how should you represent a word in a computer? 
You could store its ascii character representation, but that only tells you what the word *is*, it doesn't say much about what it *means* (you might be able to derive its part of speech from its affixes, or properties from its capitalization, but not much). 
Even more, in what sense could you combine these representations? 
We often want dense outputs from our neural networks, where the inputs are $|V|$ dimensional, where $V$ is our vocabulary, but often the outputs are only a few dimensional (if we are only predicting a handful of labels, for instance). 
How do we get from a massive dimensional space to a smaller dimensional space?
-->

# 単語埋め込みモデル 語彙的意味の符号化

単語埋め込みモデルとは，実数ベクトルであり、語彙中の単語毎に 1 つのベクトルが割り当てられます。
自然言語処理 (NLP: Natural Language Processings) では，ほとんどの場合，処理すべき特徴は単語であることが多いです。
ではコンピュータではどのように単語を表現すればよいのでしょうか？
アスキー文字の表現を保存することはできます。
ですが，文字はその単語が何であるかを教えてくれるだけです。
その単語の意味についてはは，さほど意味を持ちません。
(接辞情報から品詞を導き出すことは可能でしょう。
大文字化してある単語などからその語彙特性を導き出すことも可能でしょう。
ですが，そのような特徴にはあまり意味がありません)。
さらに、どのような意味でこれらの表現を組み合わせることができるのでしょうか？
ニューラルネットワークからの出力は、入力が $|V|$ 次元となります。
(訳注: NLP では語彙辞書に登録されている語彙数を表す表記として $||$ を用いる習慣がある)
$V$ は語彙辞書を表します。ですが，出力は数次元であることが多いです。
例えばラベルの一部だけを予測する場合などです。
(訳注: NLP では出現頻度の少ない単語を OOV (Out Of Vocabulary) として一括してラベル `<unk>` などを割り当てて
しまうことが行われる。
これを OOV 問題などという。
`<unk>` ラベルを割り当てることにより，極端に学習時間が必要な低頻度語の処理を無視して効率向上を意図する場合がある。
同様にして，具体的な数字なども 一括して同一ラベル，例えば '<num>` あるいは `<digit>` などにする。
実数以上の数概念は概念上無限に存在するので，各数値にラベルを割り振ることは効率的でないことから，
このような前処理が行われる
)
どうやって巨大な次元空間から小さな次元空間に移動させのでしょうか？

<!--How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by-->

アスキー表現の代わりに、ワンホットエン表現を使うのはどうでしょうか？
つまり $w$ という単語を以下のようにします:

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

<!--where the 1 is in a location unique to $w$. Any other word will have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how huge it is. It basically treats all words as independent entities with no relation to each other. 
What we really want is some notion of *similarity* between words. Why? Let's see an example.

Suppose we are building a language model. Suppose we have seen the sentences
-->

ここで 1 は単語 $w$ に固有の場所になります。
他の単語はどこか他の場所に 1 があり，それ以外の場所には 0 があります。
(訳注: すなわち ワンホットベクトルとは，いずれか一つの要素だけが 1 で他の要素はすべて 0 であるベクトルを言う。
ビショップの有名な教科書 PAML までは 1-of-k 表現と読んでいた。
だが最近では，ワンホット表現と呼ぶようになってきている。)

このワンホット表現には，その巨大さ以外にも大きな欠点があります。
この表現は基本的に、すべての単語を互いに関係のない独立した実体として扱います。

私たちが本当に必要としているのは、単語間の *類似性* という概念です。
なぜでしょうか？例を見てみましょう。

言語モデルを構築しているとしましょう。次のような文章を見た (言語モデルに与えられた) とします。

<!--
* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before seen in our training data:

* The physicist solved the open problem.
-->

- その数学者は店に駆け込んだ。
- その物理学者は店に駆け込んだ。
- その数学者はその未解決問題を解いた。

ここで，学習データ中には存在しない新しい文が出てきたとしましょう

- その物理学者はその未解決問題を解いた。

我々人間の持っている言語モデルでは，この文でも問題ないかもしれません。
ですがが，次の 2 つの事実を使うことができればもっと良いのではないでしょうか？

- 我々は，ある文の中で数学者と物理学者が同じ役割を果たしているのを見たことがある。どういうわけか，彼らは意味的な関係を持っている。
- 我々は，今，物理学者を見ているのと同じように，この新しい文中で数学者を同じ役割で見てきた，と推論して，
物理学者が実はこの新しい見たことのない文の中で同じ役割を果たしている

これが類似性の概念の意味です。
単に類似した正書法表現を持つことではなく，意味的な類似性を意味しています。
これは，見たことのあるものと見ていないものの間の点を結びつけることで，言語データの乏しさに対抗するための技法です。
この例はもちろん、言語学的な基本的な仮定に依存しています：類似した文脈に登場する単語は、意味的に互いに関連しているということです。これは [分布仮説](https://en.wikipedia.org/wiki/Distributional_semantics) と呼ばれています。

<!--
Our language model might do OK on this sentence, but wouldn't it be much better if we could use the following two facts:

* We have seen  mathematician and physicist in the same role in a sentence. Somehow they have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen sentence? 
This is what we mean by a notion of similarity: we mean *semantic similarity*, not simply having similar orthographic representations. 
It is a technique to combat the sparsity of linguistic data, by connecting the dots between what we have seen and what we haven't. 
This example of course relies on a fundamental linguistic assumption: that words appearing in similar contexts are related to each other semantically. 
This is called the [`distributional hypothesis`](https://en.wikipedia.org/wiki/Distributional_semantics).
-->

<!--
### Getting Dense Word Embeddings

How can we solve this problem? That is, how could we actually encode semantic similarity in words? 
Maybe we think up some semantic attributes. 
For example, we see that both mathematicians and physicists can run, so maybe we give these words a high score for the "is able to run" semantic attribute. Think of some other attributes, and imagine what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector, like this:
-->

### 密な単語埋め込み表現を得る

この問題を解決するにはどうすればよいのでしょうか。
すなわち，どのようにすれば，実際に単語の意味的類似性を符号化することができるのでしょうか？
意味的な属性を考えてみましょう。
例えば，数学者も物理学者も走ることができるので，「走ることができる」という意味的属性に高いスコアを与えることができるかもしれません。
他の属性について考えてみて，それらの属性に共通する単語にどのようなスコアをつけるかを想像してみてください。

各属性に次元が割当可能であれば，各単語には次のようなベクトルを付与できるでしょう:

\begin{align}q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]\end{align}

\begin{align}q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]\end{align}

<!--Then we can get a measure of similarity between these words by doing:-->
こうすれば，単語間の類似性の尺度を得ることができます:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}\end{align}

<!--Although it is more common to normalize by the lengths:-->
一般には，長さで正規化します:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

<!--Where $\phi$ is the angle between the two vectors. That way, extremely similar words (words whose embeddings point in the same direction) will have similarity 1. Extremely dissimilar words should have similarity -1.-->

ここで $\phi$ は 2 つのベクトルの間の角度です。
このようにして 極端に似ている単語 (埋め込みベクトルが同じ方向を向いている単語） は 類似度が 1 になります。
極めて似ていない単語は類似度が -1 になります。

<!--You can think of the sparse one-hot vectors from the beginning of this section as a special case of these new vectors we have defined, where each word basically has similarity 0, and we gave each word some unique semantic attribute. 
These new vectors are *dense*, which is to say their entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of different semantic attributes that might be relevant to determining similarity, and how on earth would you set the values of the different attributes? Central to the idea of deep learning is that the neural network learns representations of the features, rather than requiring the programmer to design them herself. 
So why not just let the word embeddings be parameters in our model, and then be updated during training? 
This is exactly what we will do. We will have some *latent semantic attributes* that the network can, in principle, learn. 
Note that the word embeddings will probably not be interpretable. 
That is, although with our hand-crafted vectors above we can see that mathematicians and physicists are similar in that they both like coffee, if we allow a neural network to learn the embeddings and see that both mathematicians and physicists have a large value in the second dimension, it is not clear what that means. 
They are similar in some latent semantic dimension, but this probably has no interpretation to us.
-->

本節冒頭にある 疎なワンホットベクトルは，我々が定義したこれらの新しいベクトルの特殊な場合と考えることができます。
これらの新しいベクトルは *密 dense* です。
すなわち，ベクトルの各要素は (典型的には) 非ゼロであることを意味します。

しかし、これらの新しいベクトルには大きな問題があります:
類似度を決定するのに関連する何千もの異なる意味属性を考えることができます。
一体どうやって異なる属性の値を設定するのでしょうか？
ディープラーニングの考え方の中心にあるのは，プログラマが自分で特徴を設計するのではなく，
ニューラルネットワークが特徴表現を学習するということです。
では、単語の埋め込みをモデルのパラメータにして、学習中に更新するのはどうでしょうか？
これをまさに我々は行おうとしています。
ネットワークが原理的に学習できるいくつかの *潜在意味属性 latent semantic attributes* を持つことになります。
単語の埋め込みベクトルは，おそらく解釈できないことに注意してください。
つまり，上記の手作りのベクトルでは，数学者と物理学者はコーヒーが好きという点で似ていることがわかります。
ですが，ニューラルネットワークに埋め込みベクトルを学習させて，数学者と物理学者の両方が 2 次元で大きな値を持っていることがわかったとしても，それが何を意味するのかはわかりません。
彼らは何らかの潜在的な意味的次元で似ているが、これはおそらく我々には何の解釈も与えません。

<!--In summary, **word embeddings are a representation of the *semantics* of a word, efficiently encoding semantic information that might be relevant to the task at hand**. 
You can embed other things too: part of speech tags, parse trees, anything! The idea of feature embeddings is central to the field.-->

要約すると、**単語埋め込みモデルとは 単語の *意味* を表現したモデルである。このモデルは当該課題に関連する意味情報を効率的に符号化する**。
他のものも埋め込むことができます: 音声タグの一部、構文解析木など，何でもいいのです。
特徴埋め込みモデルのアイデアは，この分野の中心的なものです。

### Word Embeddings in Pytorch

Before we get to a worked example and an exercise, a few quick notes about how to use embeddings in Pytorch and in deep learning programming in general. 
Similar to how we defined a unique index for each word when making one-hot vectors, we also need to define an index for each word when using embeddings. 
These will be keys into a lookup table. That is, embeddings are stored as a $|V| \times D$ matrix, where $D$ is the dimensionality of the embeddings, such that the word assigned index $i$ has its embedding stored in the $i$'th row of the matrix. 
In all of my code, the mapping from words to indices is a dictionary named word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding, which takes two arguments: the vocabulary size, and the dimensionality of the embeddings.

To index into this table, you must use torch.LongTensor (since the indices are integers, not floats).

### Pytorch での単語埋め込みモデル

作業例と演習に入る前に，Pytorch や一般的な深層学習プログラミングでの埋め込みモデル の使用方法について簡単なメモをしておきます。
ワンホットベクトルを作るときに各単語にユニークなインデックスを定義したのと同じように，単語埋め込みモデルを使うときにも各単語にインデックスを定義する必要があります。
これらはルックアップテーブルへのキーとなります。
つまり、埋め込み表現は $|V| \times D$ 行列として格納されます。
ここで $D$ は埋め込みの次元数，インデックス $i$ が割り当てられた単語は，その埋め込みが行列の $i$' 番目の行に格納されます。
以下のコードでは，単語からインデックスへのマッピングは word\_to\_ixと いう名前の辞書になっています。

埋め込みを使用するためのモジュールは ``torch.nn.Embedding`` で 2 つの引数をとります。

このテーブルにインデックスを作成するには ``torch.LongTensor`` を使用しなければなりません 
(インデックスは浮動小数点ではなく整数)。



In [2]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 つの単語を 5 次元の埋め込みモデルとして表現
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


### 例: N-グラム言語モデル

<!--### An Example: N-Gram Language Modeling-->

<!--Recall that in an n-gram language model, given a sequence of words $w$, we want to compute-->
N-Gram 言語モデルでは，与えられた単語列 $w$ を次のように表現します:

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

ここで $w_i$ は系列位置 $i$ の単語を表します。
<!--Where $w_i$ is the ith word of the sequence.-->

<!--
In this example, we will compute the loss function on some training examples and update the parameters with backpropagation.
-->

本例では，訓練データに基づいて損失関数を計算し，誤差逆伝播法でパラメータを更新します


In [8]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# シェイクスピアのソネット 2 を使います
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# 訳注: 徒然草 冒頭へ変更
test_sentence = """つれづれ なる まま に 日暮らし 硯 に むかひ て 心 に うつり ゆく よしなし ごと を そこはかとなく 書き つくれ ば あやしう こそ ものぐるほし け れ""".split()
# 入力をトークン化する必要がありますが、今のところは無視して
# タプルのリストを作成します。各タプルは ([ word_i-2, word_i-1 ], target word) です.
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# 最初の 3 つを表示します。どうなっているかを確認してください
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # ステップ 1. モデルに渡す入力を準備します (すなわち、単語を整数インデックスに
        # 変換してテンソルにします)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # ステップ2. PyTorch は勾配を蓄積することを思い出してください 
        # 新しいインスタンスに渡す前に，古いインスタンスの勾配を ゼロで 
        # 初期化する必要があります。
        model.zero_grad()

        # ステップ 3. データの前向き伝播，次単語の対数確率を計算します
        log_probs = model(context_idxs)

        # ステップ 4. 損失関数の計算します (PyTorch では目標単語をテンソル化して渡します)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # ステップ 5. 誤差勾配の逆向き伝播
        loss.backward()
        optimizer.step()

        # tensor.item() を呼び出すことで 要素が 1 であるテンソルから数値を取り出す
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # 損失値は訓練の繰り返し時に減少する

[(['つれづれ', 'なる'], 'まま'), (['なる', 'まま'], 'に'), (['まま', 'に'], '日暮らし')]
[73.11203050613403, 72.59552907943726, 72.08381295204163, 71.57616806030273, 71.0731291770935, 70.57366609573364, 70.07831168174744, 69.58672142028809, 69.09947919845581, 68.61528730392456]


<!--
### Exercise: Computing Word Embeddings: Continuous Bag-of-Words

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep learning. It is a model that tries to predict words given the context of a few words before and a few words after the target word. 
This is distinct from language modeling, since CBOW is not sequential and does not have to be probabilistic. 
Typcially, CBOW is used to quickly train word embeddings, and these embeddings are used to initialize the embeddings of some more complicated model. Usually, this is referred to as *pretraining embeddings*. It almost always helps performance a couple of percent.
-->

### 演習 単語埋め込みベクトルの計算。CBOW モデル

NLP のディープラーニングでは CBOW（Continuous Bag-of-Word）モデルがよく使われています。
CBOW モデルは，対象となる単語の前の数語と後の数語の文脈を与えられた単語を予測しようとするモデルです。
CBOW が逐次的ではなく、確率的である必要がないため、言語モデルとは異なります。
通常 CBOW は単語埋め込みを素早く訓練するために使用されます。
この単語埋め込みモデルは，より複雑なモデルの埋め込み表現を初期化するために使用さます。
通常 このことを *事前埋め込み pretraining embeddings* と呼びます。
これにより，ほとんどの場合，パフォーマンスが数パーセント向上します。

<!--
The CBOW model is as follows. Given a target word $w_i$ and an $N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$ and $w_{i+1}, \dots, w_{i+N}$, referring to all context words collectively as $C$, CBOW tries to minimize
-->

CBOW モデルは次のとおりです:
目標誤 $w_i$ と 目標語の前後を表す内容語 $C$ が幅 $N$ のウィンドウで与えられます。w_{i-1}, \dots, w_{i-N}$ and $w_{i+1}, \dots, w_{i+N}$, CBOW モデルは以下の目的関数を最小化させます:

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

<!--where $q_w$ is the embedding for word $w$.-->
ここで $q_w$ は 単語 $w$ の埋め込みベクトルです。

このモデルを PyTorc で実装するには以下のクラスを必要とします。
ヒントをいくつか以下に: 

<!--
Implement this model in Pytorch by filling in the class below. Some tips:

* Think about which parameters you need to define.
* Make sure you know what shape each operation expects. Use .view() if you need to reshape.
-->

* どのようなパラメータを定義する必要があるか考えてください。
* 各操作がどのような形を想定しているかを確認してください。reshape が必要な場合は .view() を使用してください。





In [10]:
CONTEXT_SIZE = 2  # 前後の 2 単語を内容語とします
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


tensor([ 9, 30,  3,  1])